Pipeline:
1. merge_samples
	-merge_peaks
	-annotate_peaks
	-bash: homerTools extract
	-bash: makeTagDirectory
	-make_bedgraph_file
2. peaks_break_into_files

3. peaks_to_annotation.ipynb

4. Merge_into_tissues

5. retrieve_tss_peaks

6. Rerun Figures2

7. ATAC: Create species 

8. ATAC: run pipeline

9. ATAC: Copy folders

10. Separate peaks based on expression  (needed?)

11. create_output

12. Rerun Figure3

13. Tag Histograms Compare Experimental to RefSeq

14. Nucleotide Frequency

15. Rerun  CDF of TSS with Samples.ipynb

16. Compare RNA_seq

17. Igv snapshot


## Wrapper on merge_peaks and anno_peaks

In [1]:
parameter_file = 'params/params.yaml'
import yaml
import sys
import glob
import os
import pandas as pd
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
#annotation=doc['annotation_gtf']
annotation=doc['annotation']
from Homer import *

In [2]:
meta_files = pd.read_csv("../raw_metaSamples.tsv",sep='\t',index_col=0)
meta_files

,Tissue,Experiment,Name,Full
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC1,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC2,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,BMDM1hKLA,GRO,BMDM1hKLA_GRO1,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,BMDM1hKLA,GROCap,BMDM1hKLA_GROCap1,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,BMDMwt,ATAC,BMDMwt_ATAC1,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...
CHBMDMVehic1_ATAC_JHS997_SD_GTGTGGTG_S8_L001_R1_001.fastq.gz,BMDMwt,ATAC,BMDMwt_ATAC2,CHBMDMVehic1_ATAC_JHS997_SD_GTGTGGTG_S8_L001_R...
CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq.gz,BMDMwt,GRO,BMDMwt_GRO1,CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001...
CHBMDMwt_GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001.fastq.gz,BMDMwt,GROCap,BMDMwt_GROCap1,CHBMDMwt_GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001...
CHBrain_ATAC_JHS992_SD_ACCACTGT_S4_L001_R1_001.fastq.gz,Brain,ATAC,Brain_ATAC1,CHBrain_ATAC_JHS992_SD_ACCACTGT_S4_L001_R1_001...
CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq.gz,Brain,GRO,Brain_GRO1,CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001....


In [3]:
all_peak_merge_files = glob.glob("f04_peaks/*")
print(all_peak_merge_files)

['f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1', 'f04_peaks/Liver_START2_and_Liver_startinput1', 'f04_peaks/Brain_GROCap1_and_Brain_GRO1', 'f04_peaks/Liver_START1_and_Liver_startinput1', 'f04_peaks/Lung_GROCap1_and_Lung_GRO1', 'f04_peaks/Brain_START3_and_Brain_startinput1', 'f04_peaks/Heart_START1_and_Heart_startinput1', 'f04_peaks/CHO_GROCap2_and_CHO_GRO1', 'f04_peaks/Heart_START2_and_Heart_startinput1', 'f04_peaks/Liver_GROCap1_and_Liver_GRO1', 'f04_peaks/Pancreas_START2_and_Pancreas_startinput1', 'f04_peaks/BMDM1hKLA_GROCap1_and_BMDM1hKLA_GRO1', 'f04_peaks/Pancreas_START1_and_Pancreas_startinput1', 'f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_startinput1', 'f04_peaks/Muscle_START1_and_Muscle_startinput1', 'f04_peaks/Brain_START1_and_Brain_startinput1', 'f04_peaks/Muscle_START2_and_Muscle_startinput1', 'f04_peaks/Brain_START2_and_Brain_startinput2', 'f04_peaks/Brain_START1_and_Brain_startinput2', 'f04_peaks/CHO_GROCap1_and_CHO_GRO2', 'f04_peaks/Brain_START2_a

### Merge

In [4]:
if not os.path.exists("Results"):
    os.mkdir("Results/")
if not os.path.exists("Results/merged/"):
    os.mkdir("Results/merged")

In [5]:
output_file = 'Results/merged/samples.merge'                

In [6]:
merge_peaks(all_peak_merge_files,output_file,dist='given')

mergePeaks -d given -venn Results/merged/samples.merge_venn -matrix Results/merged/samples.merge_mat -strand f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1 f04_peaks/Liver_START2_and_Liver_startinput1 f04_peaks/Brain_GROCap1_and_Brain_GRO1 f04_peaks/Liver_START1_and_Liver_startinput1 f04_peaks/Lung_GROCap1_and_Lung_GRO1 f04_peaks/Brain_START3_and_Brain_startinput1 f04_peaks/Heart_START1_and_Heart_startinput1 f04_peaks/CHO_GROCap2_and_CHO_GRO1 f04_peaks/Heart_START2_and_Heart_startinput1 f04_peaks/Liver_GROCap1_and_Liver_GRO1 f04_peaks/Pancreas_START2_and_Pancreas_startinput1 f04_peaks/BMDM1hKLA_GROCap1_and_BMDM1hKLA_GRO1 f04_peaks/Pancreas_START1_and_Pancreas_startinput1 f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_startinput1 f04_peaks/Muscle_START1_and_Muscle_startinput1 f04_peaks/Brain_START1_and_Brain_startinput1 f04_peaks/Muscle_START2_and_Muscle_startinput1 f04_peaks/Brain_START2_and_Brain_startinput2 f04_peaks/Brain_START1_and_Brain_startinput2 f04_peaks/CH

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Annotate

In [7]:
output_anno_file = 'Results/merged/samples.merge.anno' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file,ref_fa=ref_fa,annotation=annotation)

annotatePeaks.pl Results/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3 > Results/merged/samples.merge.anno


### Extract sequences

In [8]:
f_out = 'Results/merged/samples.merge.anno.fa'
cmd = 'homerTools extract {f_in} {ref_fa} -fa > {f_out}'.format(f_in=output_anno_file,ref_fa=ref_fa,f_out=f_out)
!{cmd}


	Extracting sequences from file: /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna
	Looking for peak sequences in a single file (/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna)
	Extracting 1547 sequences from NW_020822366.1
	Extracting 8450 sequences from NW_020822370.1
	Extracting 1435 sequences from NW_020822372.1
	Extracting 1697 sequences from NW_020822373.1
	Extracting 936 sequences from NW_020822374.1
	Extracting 7343 sequences from NW_020822375.1
	Extracting 1590 sequences from NW_020822376.1
	Extracting 1204 sequences from NW_020822377.1
	Extracting 1257 sequences from NW_020822379.1
	Extracting 1747 sequences from NW_020822381.1
	Extracting 739 sequences from NW_020822382.1
	Extracting 547 sequences from NW_020822385.1
	Extracting 1252 sequences from NW_020822388.1
	Extracting 513 sequences from NW_020822390.1
	Extracting 140 sequences from NW_020822391.1
	Extracting 711 sequences from NW_020822393.1
	Extracting 284 seque

	Extracting 6 sequences from NW_020822609.1
	Extracting 4363 sequences from NW_020822610.1
	Extracting 5987 sequences from NW_020822611.1
	Extracting 2922 sequences from NW_020822614.1
	Extracting 14964 sequences from NW_020822615.1
	Extracting 3776 sequences from NW_020822616.1
	Extracting 1 sequences from NW_020822617.1
	Extracting 1818 sequences from NW_020822618.1
	Extracting 1237 sequences from NW_020822619.1
	Extracting 997 sequences from NW_020822620.1
	Extracting 497 sequences from NW_020822621.1
	Extracting 884 sequences from NW_020822623.1
	Extracting 279 sequences from NW_020822626.1
	Extracting 185 sequences from NW_020822627.1
	Extracting 219 sequences from NW_020822628.1
	Extracting 3435 sequences from NW_020822629.1
	Extracting 4422 sequences from NW_020822630.1
	Extracting 2067 sequences from NW_020822632.1
	Extracting 3080 sequences from NW_020822634.1
	Extracting 1 sequences from NW_020822635.1
	Extracting 2650 sequences from NW_020822636.1
	Extracting 2485 sequences 

	Extracting 1 sequences from NW_020823241.1
	Extracting 2 sequences from NW_020823243.1
	Extracting 1 sequences from NW_020823246.1
	Extracting 1 sequences from NW_020823252.1
	Extracting 6 sequences from NW_020823257.1
	Extracting 5 sequences from NW_020823258.1
	Extracting 1 sequences from NW_020823280.1
	Extracting 6 sequences from NW_020823286.1
	Extracting 1 sequences from NW_020823295.1
	Extracting 2 sequences from NW_020823296.1
	Extracting 1 sequences from NW_020823298.1
	Extracting 2 sequences from NW_020823300.1
	Extracting 1 sequences from NW_020823304.1
	Extracting 1 sequences from NW_020823306.1
	Extracting 1 sequences from NW_020823310.1
	Extracting 9 sequences from NW_020823318.1
	Extracting 3 sequences from NW_020823326.1
	Extracting 2 sequences from NW_020823327.1
	Extracting 1 sequences from NW_020823334.1
	Extracting 1 sequences from NW_020823361.1
	Extracting 1 sequences from NW_020823364.1
	Extracting 1 sequences from NW_020823372.1
	Extracting 1 sequences from NW_

	Extracting 2 sequences from NW_020823700.1
	Extracting 27 sequences from NW_020823701.1
	Extracting 1 sequences from NW_020823702.1
	Extracting 2 sequences from NW_020823703.1
	Extracting 9 sequences from NW_020823704.1
	Extracting 4 sequences from NW_020823707.1
	Extracting 3 sequences from NW_020823710.1
	Extracting 1 sequences from NW_020823713.1
	Extracting 7 sequences from NW_020823717.1
	Extracting 1 sequences from NW_020823718.1
	Extracting 2 sequences from NW_020823720.1
	Extracting 1 sequences from NW_020823721.1
	Extracting 3 sequences from NW_020823723.1
	Extracting 2 sequences from NW_020823724.1
	Extracting 1 sequences from NW_020823725.1
	Extracting 5 sequences from NW_020823728.1
	Extracting 1 sequences from NW_020823730.1
	Extracting 2 sequences from NW_020823731.1
	Extracting 1 sequences from NW_020823732.1
	Extracting 4 sequences from NW_020823736.1
	Extracting 7 sequences from NW_020823741.1
	Extracting 1 sequences from NW_020823743.1
	Extracting 4 sequences from NW

# Merge tags

In [9]:
%%time
all_tags_start = []
for t in tissues:
    for s in ['GROCap','START']:
        #curr = glob.glob(j + '*f03_tags/trim*.peak')
        curr_raw = glob.glob(data_folder + t + '/*/')
        #print(curr_raw)
        for j in curr_raw:
            curr_type = j.split('/')[-2]  
            if curr_type == 'GROCap':
                #print(os.path.join(j,'f03_tags/*5GRO*'))
                new = glob.glob(os.path.join(data_folder,s,j,'f03_tags/*GROCap*'))
            elif curr_type == 'START':
                new = np.array(glob.glob(os.path.join(data_folder,s,j,'f03_tags/*')))
                if not len(new) == 0:
                    new = (new[(np.array(map(lambda x: 'input' not in x,new)))]) #Remove the input
            else:
                continue
            all_tags_start.extend(new)



curr_out = 'Results/merged/tags_TSS_merged'
cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=curr_out,files = ' '.join(all_tags_start))
print(cmd)
!{cmd}

makeTagDirectory Results/merged/tags_TSS_merged -single -d /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1 /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1 /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDMwt/GROCap/f03_tags/trim_BMDMwt_GROCap1 /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDMwt/GROCap/f03_tags/trim_BMDMwt_GROCap1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/GROCap/f03_tags/trim_Brain_GROCap1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START3 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START2 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/GROCap/f03_tags/trim_Brain_GROCap1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START3 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START1 /data/i

	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/GROCap/f03_tags/trim_BMDM1hKLA_GROCap1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDMwt/GROCap/f03_tags/trim_BMDMwt_GROCap1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDMwt/GROCap/f03_tags/trim_BMDMwt_GROCap1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/GROCap/f03_tags/trim_Brain_GROCap1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START3
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START2
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/GROCap/f03_tags/trim_Brain_GROCap1
	Adding tag directory /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_START3
	Adding tag directory /data/isshamie/TSS/N

In [10]:
%%time
all_tags_start = []
for t in tissues:
    for s in ['GROCap','START']:
        #curr = glob.glob(j + '*f03_tags/trim*.peak')
        j = glob.glob(os.path.join(data_folder, t,s))
        if len(j) > 0:
            j = j[0]
        else:
            continue
        curr_type = s
        if curr_type == 'GROCap':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = np.array(glob.glob(os.path.join(data_folder,s,j,'f03_tags/*GRO*')))
            if not len(new) == 0:
                new = (new[(np.array(map(lambda x: 'Cap' not in x,new)))]) #Remove the input
        elif curr_type == 'START':
            new = np.array(glob.glob(os.path.join(data_folder,s,j,'f03_tags/*input*')))
        else:
            continue
        all_tags_start.extend(new)



curr_out = 'Results/merged/tags_input_merged'
cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=curr_out,files = ' '.join(all_tags_start))
print(cmd)
!{cmd}

makeTagDirectory Results/merged/tags_input_merged -single -d /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_STARTinput2 /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/START/f03_tags/trim_Brain_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/FemaleReproductive/START/f03_tags/trim_FemaleReproductive_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Heart/START/f03_tags/trim_Heart_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Kidney/START/f03_tags/trim_Kidney_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Liver/START/f03_tags/trim_Liver_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Lung/START/f03_tags/trim_Lung_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/MiscOrgans/START/f03_tags/trim_MiscOrgans_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Muscle/START/f03_tags/trim_Muscle_STARTinput1 /data/isshamie/TSS/NCBI_PICR_103_Processed/Pancreas/START/f03_tags/trim_Pancreas_STARTinput1 /data/isshamie/TSS/NCBI_P

In [11]:
def make_bedgraph_file(in_dir,out_file):
    ''' Create bedgraph file for experiment. - strand will have negative values to be used'''
    
    out_pos = os.path.join(os.path.dirname(out_file),  "pos_" + os.path.basename(out_file))
    out_neg = os.path.join(os.path.dirname(out_file),  "neg_" + os.path.basename(out_file))
    
    cmd = ('makeUCSCfile {in_dir} -o {out_file} -strand + -fragLength 1').format(in_dir=in_dir,out_file=out_pos)
    print(cmd);sys.stdout.flush()
    sarge.run(cmd)

    cmd = ('makeUCSCfile {in_dir} -o {out_file} -strand - -fragLength 1').format(in_dir=in_dir,out_file=out_neg)
    print(cmd);sys.stdout.flush()
    sarge.run(cmd)
    print('here')
    print(out_file)
    cmd = "gunzip -c %s.gz | awk '{$4=-$4; print}' | gzip > %s_tmp.gz" % (out_neg,out_neg)# .format(out_f_neg=out_neg) #Change - strand to have neg values
    print(cmd)
    sarge.run(cmd)
    out = out_pos.replace('_pos','') #trim_CHO--mSTART-JHS823_S21_R1_001_pos.bedgraph.gz
    
    cmd = ("cat {out_f_0}.gz {out_f_1}_tmp.gz > {final}.gz").format(out_f_0=out_pos,out_f_1=out_neg,final=out_file) #Concatenate the 2 into new merged file
    print(cmd)
    sarge.run(cmd)
    cmd = ("rm {out_f_0}.gz {out_f_1}_tmp.gz {out_f_1}.gz").format(out_f_0=out_pos,out_f_1=out_neg) #replace pos and neg
    print(cmd)
    sarge.run(cmd)
    return

In [12]:
in_dir = "Results/merged/tags_input_merged"
out_file = "Results/merged/tags_input_merged.bedgraph"

make_bedgraph_file("Results/merged/tags_input_merged",out_file)

makeUCSCfile Results/merged/tags_input_merged -o Results/merged/pos_tags_input_merged.bedgraph -strand + -fragLength 1
makeUCSCfile Results/merged/tags_input_merged -o Results/merged/neg_tags_input_merged.bedgraph -strand - -fragLength 1
here
Results/merged/tags_input_merged.bedgraph
gunzip -c Results/merged/neg_tags_input_merged.bedgraph.gz | awk '{$4=-$4; print}' | gzip > Results/merged/neg_tags_input_merged.bedgraph_tmp.gz
cat Results/merged/pos_tags_input_merged.bedgraph.gz Results/merged/neg_tags_input_merged.bedgraph_tmp.gz > Results/merged/tags_input_merged.bedgraph.gz
rm Results/merged/pos_tags_input_merged.bedgraph.gz Results/merged/neg_tags_input_merged.bedgraph_tmp.gz Results/merged/neg_tags_input_merged.bedgraph.gz


In [13]:
in_dir = "Results/merged/tags_TSS_merged"
out_file = "Results/merged/tags_TSS_merged.bedgraph"

make_bedgraph_file("Results/merged/tags_TSS_merged",out_file)

makeUCSCfile Results/merged/tags_TSS_merged -o Results/merged/pos_tags_TSS_merged.bedgraph -strand + -fragLength 1
makeUCSCfile Results/merged/tags_TSS_merged -o Results/merged/neg_tags_TSS_merged.bedgraph -strand - -fragLength 1
here
Results/merged/tags_TSS_merged.bedgraph
gunzip -c Results/merged/neg_tags_TSS_merged.bedgraph.gz | awk '{$4=-$4; print}' | gzip > Results/merged/neg_tags_TSS_merged.bedgraph_tmp.gz
cat Results/merged/pos_tags_TSS_merged.bedgraph.gz Results/merged/neg_tags_TSS_merged.bedgraph_tmp.gz > Results/merged/tags_TSS_merged.bedgraph.gz
rm Results/merged/pos_tags_TSS_merged.bedgraph.gz Results/merged/neg_tags_TSS_merged.bedgraph_tmp.gz Results/merged/neg_tags_TSS_merged.bedgraph.gz


In [14]:
import IPython
print IPython.sys_info()

{'commit_hash': u'5a894b9',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython',
 'ipython_version': '5.3.0',
 'os_name': 'posix',
 'platform': 'Linux-4.4.0-130-generic-x86_64-with-debian-stretch-sid',
 'sys_executable': '/home/isshamie/software/anaconda2/bin/python',
 'sys_platform': 'linux2',
 'sys_version': '2.7.13 |Anaconda custom (64-bit)| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'}


In [15]:
%load_ext version_information
%version_information pandas, numpy, seaborn


Software versions
Python 2.7.13 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.3.0
OS Linux 4.4.0 130 generic x86_64 with debian stretch sid
pandas 0.23.1
numpy 1.13.3
seaborn 0.7.1
Mon Jan 21 15:31:07 2019 PST

In [18]:
#find the names of the imported modules
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

#exclude all modules not listed by `!pip freeze`
excludes = ['__builtin__', 'types', 'IPython.core.shadowns', 'sys', 'os']
imported_modules = [module for module in imports() if module not in excludes]
pip_modules = !pip freeze #you could also use `!conda list` with anaconda
conda_modules = !conda list #you could also use `!conda list` with anaconda


pip_modules
# #print the names and versions of the imported modules
# for module in pip_modules:
#     name, version = module.split('==')
#     if name in imported_modules:
#         print(name + '\t' + version)

        
# for module in conda_modules:
#     name, version = module.split('==')
#     if name in imported_modules:
#         print(name + '\t' + version)


['You are using pip version 9.0.1, however version 18.1 is available.',
 "You should consider upgrading via the 'pip install --upgrade pip' command.",
 'alabaster==0.7.10',
 'anaconda-client==1.6.3',
 'anaconda-navigator==1.6.2',
 'anaconda-project==0.6.0',
 'asn1crypto==0.22.0',
 'astroid==1.4.9',
 'astropy==2.0.7',
 'Babel==2.4.0',
 'backports-abc==0.5',
 'backports.functools-lru-cache==1.5',
 'backports.shutil-get-terminal-size==1.0.0',
 'backports.ssl-match-hostname==3.4.0.2',
 'beautifulsoup4==4.6.0',
 'biopython==1.72',
 'bitarray==0.8.1',
 'blaze==0.10.1',
 'bleach==1.5.0',
 'bokeh==0.12.5',
 'boto==2.46.1',
 'Bottleneck==1.2.1',
 'brewer2mpl==1.4.1',
 'bz2file==0.98',
 'cdecimal==2.3',
 'certifi==2018.8.24',
 'cffi==1.10.0',
 'chardet==3.0.3',
 'Circos==1.3.5',
 'click==6.7',
 'cloudpickle==0.2.2',
 'clyent==1.2.2',
 'colorama==0.3.9',
 'colormath==2.1.1',
 'conda==4.5.11',
 'configparser==3.5.0',
 'contextlib2==0.5.5',
 'cooler==0.7.6',
 'cryptography==1.8.1',
 'cutadapt==1.16

In [19]:
conda_modules

['# packages in environment at /home/isshamie/software/anaconda2:',
 '#',
 '# Name                    Version                   Build  Channel',
 '_license                  1.1                      py27_1  ',
 'alabaster                 0.7.10                   py27_0  ',
 'anaconda                  custom           py27h4a00acb_0  ',
 'anaconda-client           1.6.3                    py27_0  ',
 'anaconda-navigator        1.6.2                    py27_0  ',
 'anaconda-project          0.6.0                    py27_0  ',
 'argparse                  1.4.0                     <pip>',
 'asn1crypto                0.22.0                   py27_0  ',
 'astroid                   1.4.9                    py27_0  ',
 'astropy                   2.0.7                    py27_1    conda-forge',
 'babel                     2.4.0                    py27_0  ',
 'backports                 1.0                      py27_0  ',
 'backports.functools_lru_cache 1.5                      py27_0    conda-for